In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.keras import utils
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback

import tensorflow as tf

import numpy as np
import pandas as pd

import nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('punkt')

#Codes by Erin Ward https://www.kaggle.com/eward96/derry-girls-new-script-generator

In [ ]:
#Codes by Erin Ward https://www.kaggle.com/eward96/derry-girls-new-script-generator

def process(text):
    with open (text, "r", encoding = "ISO-8859-1") as file:
        data=file.readlines()
        script = ""
        for i in data:
            i = i.lower().replace('"', '').replace("\n", " \n ")
            if i.strip() != "":
                script += "".join(i).replace("\n"," \n ")
        return script

In [ ]:
df = process("../input/youtube-videos-comments-analysis/russian_soldiers.csv") 
print(df[0:2000])

In [ ]:
Text_Data = df

charindex = list(set(Text_Data))
charindex.sort() 
print(charindex)

np.save("charindex.npy", charindex)

print(len(Text_Data))

In [ ]:
chars_size = len(charindex)
seq_len = 80

x_train = []
y_train = []

for i in range(0, len(Text_Data)-seq_len, 1 ): 
    X = Text_Data[i:i + seq_len]
    Y = Text_Data[i + seq_len]
    x_train.append([charindex.index(x) for x in X])
    y_train.append(charindex.index(Y))

x_train = np.reshape(x_train, (len(x_train), seq_len))

y_train = utils.to_categorical(y_train)

In [ ]:
#LSTMs (Long Short Term Memory networks) are excellent for learning and generating text.🏻

def LSTM_model():
    model = models.Sequential()
    inp = layers.Input(shape=(seq_len, ))
    x = layers.Embedding(chars_size, 80, trainable=False)(inp)
    x = tf.compat.v1.keras.layers.CuDNNLSTM(1024, return_sequences=True,)(x)
    x = tf.compat.v1.keras.layers.CuDNNLSTM(512, return_sequences=True,)(x)
    x = tf.compat.v1.keras.layers.CuDNNLSTM(256,)(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(256, activation="elu")(x)
    x = layers.Dense(128, activation="elu")(x)
    x = layers.Dropout(0.2)(x)
    outp = layers.Dense(chars_size, activation='softmax')(x)
    
    model = models.Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=0.0008),
                  metrics=['accuracy']
                 )

    return model

model = LSTM_model()

In [ ]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=3)

#Above error:  [[model/cu_dnnlstm/CudnnRNNV2]] [Op:__inference_train_function_3463]

#That's all for Now, since I don't know to fix it or even understand the language too.